# Universidad del Valle de Guatemala
## Security Data Science - 10
* Jose Abraham Gutierrez Corado - 19111
* Javier Alejandro Cotto Argueta - 19324

## Laboratorio 2: Detección de SPAM

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline

### Explorar Datos

In [2]:
df1 = pd.read_csv('completeSpamAssassin.csv')
df1.drop(["Unnamed: 0"], axis=1, inplace=True)
df1.sample(5)

,Body,Label
1560,JetMp3 Tï¿½rkiye'de ilk kez uygulanan bir sist...,1
1691,Norton \n SystemWorks 2002 Software S...,1
3063,empty,0
447,Classic WinesAll of this can be\n ...,1
9,"Dear ricardo1 ,\nCOST EFFECTIVE Direct Email A...",1


In [3]:
df2 = pd.read_csv('enronSpamSubset.csv')
df2.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1, inplace=True)
df2.sample(5)

,Body,Label
724,Subject: new discovery may help fight cancer\n...,1
7005,Subject: cad 9 mm\n received this afternoon by...,0
6435,Subject: continental power officialisation and...,0
3217,"Subject: re : scadsa\n dear sir ,\n greetings ...",1
4194,Subject: claim your lucky winning . . .\n winn...,1


### Merge de los df's

In [4]:
dfNew = df1.merge(df2, how="right")
dfNew.sample(5)

,Body,Label
8989,Subject: start date : 1 / 3 / 02 ; hourahead h...,0
5565,Subject: update - reimbursement of individuall...,0
1654,"Subject: localized software , all languages av...",1
6836,Subject: organizational changes\n - - - - - - ...,0
1212,Subject: this free 7 - day trial will prove th...,1


### Pre Procesamiento

In [5]:
#Conversión de Mayusculas a Minusculas
def Conv_Up_to_Low(df):
    sentences = []
    string = ""
    for sentence in df.Body:
        string = str(sentence)
        sentences.append(string.lower())
        
    df["Body"] = sentences
    
    return df

#Acentos
import unicodedata
def accents(df):
    sentences = []
    for sentence in df.Body:
        sentences.append(unicodedata.normalize('NFKD', sentence).encode('ascii','ignore').decode('utf-8','ignore'))
    
    df["Body"] = sentences
    
    return df

#Caracteres especiales, números y simbolos
import re
def removeSpecialCharacters(df):
    patron = r'[^a-zA-Z0-9\s]'
    sentences = []
    for sentence in df.Body:
        if patron in sentence:
            sentences.append(re.sub(patron,'', sentence))
        else:
            sentences.append(re.sub(r'[^a-zA-Z\s]', '', sentence))
    
    df["Body"] = sentences
    return df
    
#Contracciones en el texto
import contractions
def contrac(df):
    list(contractions.contractions_dict.items())[:10]
    sentences = []
    for sentence in df.Body:
        sentences.append(contractions.fix(sentence))
        
    df["Body"] = sentences
    return df

#Stop Words
import nltk
nltk.download('punkt')
nltk.download('stopwords')

def Stop_word(df):
    sentences = []
    for sentence in df.Body:
        tokens = nltk.word_tokenize(sentence.lower())
        stop_words_spanish = nltk.corpus.stopwords.words('spanish')
        tokens_filtrados = [token for token in tokens if token not in stop_words_spanish]
        stop_words_english = nltk.corpus.stopwords.words('english')
        tokens_english = nltk.word_tokenize(contractions.fix(sentence))
        tokens_filtrados_english = [token for token in tokens_english if token not in stop_words_english]
        sentences.append(tokens_filtrados_english)
        
    df["Body"] = sentences
    return df
    

[nltk_data] Downloading package punkt to C:\Users\Javier
[nltk_data]     Cotto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Javier
[nltk_data]     Cotto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
dfNew = Conv_Up_to_Low(dfNew)
dfNew = accents(dfNew)
dfNew = removeSpecialCharacters(dfNew)
dfNew = contrac(dfNew)
dfNew = Stop_word(dfNew)

In [7]:
dfNew

,Body,Label
0,"[subject, stock, promo, mover, cwtd, urgent, i...",1
1,"[subject, listed, major, search, engines, subm...",1
2,"[subject, important, information, thu, jun, su...",1
3,"[subject, utf, q, bask, life, utf, q, individu...",1
4,"[subject, bidstogo, places, go, things, hello,...",1
...,...,...
9995,"[subject, monday, nd, oct, louise, half, hour,...",0
9996,"[subject, missing, bloomberg, deals, stephanie...",0
9997,"[subject, eops, salary, survey, questionnaire,...",0
9998,"[subject, q, comparison, hi, louise, compariso...",0
